# japanese-gpt-neox-3.6b-instruction-ppo を SageMaker で Hosting
## このノートブックについて
このノートブックは、rinna の japanese-gpt-neox-3.6b-instruction-ppo モデルを、SageMaker でリアルタイム推論エンドポイントを Hosting するノートブックです。  
以下の環境で動作確認を行ってます。
* SageMaker Studio Notebooks
    * `ml.g5.2xlarge(NVIDIA A10G Tensor Core GPU 搭載 VRAM 24GB, RAM 32GB, vCPU 8)` : `PyTorch 1.13 Python 3.9 GPU Optimized`
    * `ml.m5.2xlarge(RAM 32GB, vCPU 8) ` : `PyTorch 1.13 Python 3.9 CPU Optimized`
* SageMaker Notebooks
    * `ml.g5.2xlarge(NVIDIA A10G Tensor Core GPU 搭載 VRAM 24GB, RAM 32GB, vCPU 8)` : `conda_pytorch_p39`
    * `ml.m5.2xlarge(RAM 32GB, vCPU 8) ` : `conda_pytorch_p39`  
[各インスタンスの料金についてはこちら](https://aws.amazon.com/jp/sagemaker/pricing/)をご確認ください。  

## 使用するモデルについて
モデルの詳細については[Hugging Face apanese-gpt-neox-3.6b-instruction-ppo](https://huggingface.co/rinna/japanese-gpt-neox-3.6b-instruction-ppo)  を参照してください。  
モデルのライセンスは上記リンクにあるとおり `MIT` です。

ノートブックは外部ファイルを参照していないので、どのディレクトリに配置してあっても動作します。  

また、ノートブックを動かすにあたって、各セルを上から順番に実行すれば動きますが、SageMaker 上での推論の仕組みについては、[AI/ML DarkPark](https://www.youtube.com/playlist?list=PLAOq15s3RbuL32mYUphPDoeWKUiEUhcug) の特に [Amazon SageMaker 推論 Part2すぐにプロダクション利用できる！モデルをデプロイして推論する方法 【ML-Dark-04】【AWS Black Belt】](https://youtu.be/sngNd79GpmE) をご参照ください。

## 準備
### ノートブックを動かすに当たって必要なモジュールのインストール

In [ ]:
pip install transformers==4.26 einops sagemaker SentencePiece -U

###  今回扱うモデルの動かし方について
[How to use the model](https://huggingface.co/rinna/japanese-gpt-neox-3.6b-instruction-ppo#how-to-use-the-model) に沿って実行すると動かせます。  
例えば、以下のコードをこのノートブックで実行するとテキストを生成できます。  
実行したい場合は別途セルを用意して実行してみてください。g5.2xlarge インスタンスで実行に 10 分程度かかります。(ほとんどはモデルのロード時間です)  
このノートブックでは以下のコードをベースに SageMaker で Hosting できるようにします。  
```python
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(
    'rinna/japanese-gpt-neox-3.6b-instruction-ppo', 
    use_fast=False
)
model = AutoModelForCausalLM.from_pretrained(
    'rinna/japanese-gpt-neox-3.6b-instruction-ppo'
).to("cuda")

prompt = '''ユーザー: 世界自然遺産を列挙してください。
システム: 膨大な数です。例えば国で絞ってください。
ユーザー: イギリスでお願いします。
システム:'''.replace('\n','<NL>')

token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

with torch.no_grad():
    output_ids = model.generate(
        token_ids.to(model.device),
        do_sample=True,
        max_new_tokens=128,
        temperature=0.01,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

output = tokenizer.decode(output_ids.tolist()[0][token_ids.size(1):])
output = output.replace("<NL>", "\n")
print(output)
```

### モジュール読み込み

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import gc
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFaceModel
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
import json
region = boto3.session.Session().region_name
role = sagemaker.get_execution_role()
sm = boto3.client('sagemaker')
smr = boto3.client('sagemaker-runtime')
endpoint_inservice_waiter = sm.get_waiter('endpoint_in_service')

### モデルのダウンロード
SageMaker で機械学習モデルをホスティングする際は、一般的にはモデルや推論コードなどを tar.gz の形に固めます。  
tokenizer と model を `from_pretrained` メソッドを利用してモデルをインターネットからロードして、そのままファイルをディレクトリに出力します。  

In [ ]:
# 既存のディレクトリがある場合のときのため削除
model_dir = './inference'
!rm -rf {model_dir}
!mkdir -p {model_dir}'/code'

#### tokenizer の取得と保存

In [ ]:
%%time

tokenizer = AutoTokenizer.from_pretrained(
    'rinna/japanese-gpt-neox-3.6b-instruction-ppo', 
    use_fast=False
)
tokenizer.save_pretrained(model_dir)

#### モデルの取得と保存
以下のセルは 10GB 以上のモデルを DL して保存するため 5 分ほど時間がかかります。

In [ ]:
%%time
model = AutoModelForCausalLM.from_pretrained(
    'rinna/japanese-gpt-neox-3.6b-instruction-ppo'
)
model.save_pretrained(model_dir)

モデルは SageMaker で動かすのでメモリから開放します

In [ ]:
del model
del tokenizer
gc.collect()

### 推論コードの作成
先程実行したコードをもとに記述していきます。  
まずは必要なモジュールを記述した requirements.txt を用意します。  
今回は [deep-learning-containers](https://github.com/aws/deep-learning-containers)の HuggingFace のコンテナを使います。  
einops と Sentence Piece が不足しているので requirements.txt に記載します。

In [ ]:
%%writefile inference/code/requirements.txt
einops
SentencePiece

先述のコードを SageMaker Inference 向けに改変します。
1. `model_fn` でモデルを読み込みます。先程は huggingface のモデルを直接ロードしましたが、`model_dir` に展開されたモデルを読み込みます。
2. `input_fn` で前処理を行います。
    * json 形式のみを受け付け他の形式は弾くようにします。
    * json 文字列を dict 形式に変換して返します。
3. `predict_fn` で推論します。
    1. リクエストされたテキストを token 化します。
    2. パラメータを展開します。
    3. 推論（生成）します。
    4. 生成結果をテキストにして返します。
4. `output_fn` で結果を json 形式にして返します。

In [ ]:
%%writefile inference/code/inference.py
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import json

DEVICE = 'cuda:0'

def model_fn(model_dir):
    tokenizer = AutoTokenizer.from_pretrained(
        model_dir, 
        use_fast=False
    )
    model = AutoModelForCausalLM.from_pretrained(
        model_dir
    ).to(DEVICE)
    return {'tokenizer':tokenizer,'model':model}

def input_fn(data, content_type):
    if content_type == 'application/json':
        data = json.loads(data)
    else:
        raise TypeError('content_type is only allowed application/json')
    return data

def predict_fn(data, model):
    prompt = data['prompt']
    token_ids = model['tokenizer'].encode(prompt, add_special_tokens=False, return_tensors="pt")
    do_sample = data['do_sample']
    max_new_tokens = data['max_new_tokens']
    temperature = data['temperature']
    
    with torch.no_grad():
        output_ids = model['model'].generate(
            token_ids.to(DEVICE),
            do_sample=do_sample,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            pad_token_id=model['tokenizer'].pad_token_id,
            bos_token_id=model['tokenizer'].bos_token_id,
            eos_token_id=model['tokenizer'].eos_token_id
        )
    output = model['tokenizer'].decode(output_ids.tolist()[0][token_ids.size(1):])
    output = output.replace("<NL>", "\n")
    
    return output


def output_fn(data, accept_type):
    if accept_type == 'application/json':
        data = json.dumps({'result' : data})
    else:
        raise TypeError('content_type is only allowed application/json')
    return data

### モデルアーティファクトの作成と S3 アップロード
アーティファクト(推論コード + モデル)を tar.gz に固めます。時間がかかるので `pigz` で並列処理を行います。  
ml.g5.2xlarge, ml.m5.2xlarge で 10 分ほどかかります。

※ SageMaker Studio のカーネルには pigz が入っていないので、下記 apt のセルを実行してください。SageMaker Notebooks の場合は不要です。

In [ ]:
!apt update -y
!apt install pigz -y

In [ ]:
%%time

!rm model.tar.gz
%cd {model_dir}
!tar  cv ./ | pigz -p 8 > ../model.tar.gz # 8 並列でアーカイブ
%cd ..

アーティファクトを S3 にアップロードします。60 秒程度で完了します。

In [ ]:
%%time

model_s3_uri = sagemaker.session.Session().upload_data(
    'model.tar.gz',
    key_prefix='japanese-gpt-neox-3.6b-instruction-ppo'
)
print(model_s3_uri)

## SageMaker で Hosting する
g5.2xlarge インスタンス(NVIDIA A10G Tensor Core GPU 搭載 VRAM 24GB, RAM 32GB) の場合レスポンスに 6 秒程度で済むため、リアルタイム推論エンドポイントを立てます。  
(再掲)g5.2xlarge の[料金はこちら](https://aws.amazon.com/sagemaker/pricing/?nc1=h_ls)で確認してください。  

リアルタイム推論エンドポイントを立てて推論するにあたって、SageMaker Python SDK を用いる場合と Boto3 を用いる場合の 2 パターンを紹介します。

### SageMaker Python SDKを用いる場合

#### Hosting
使用している API の詳細は以下を確認してください。  
[Amazon SageMaker Python SDK](https://sagemaker.readthedocs.io/en/stable/index.html)

##### 定数の設定

In [ ]:
model_name = 'japanese-gpt-neox-3-6b-instruction-ppo'
endpoint_config_name = model_name + 'Config'
endpoint_name = model_name + 'Endpoint'
instance_type = 'ml.g5.2xlarge'

##### 使用するコンテナイメージの URI を取得

In [ ]:
image_uri = sagemaker.image_uris.retrieve(
    framework='huggingface',
    region=region,
    version='4.26',
    image_scope='inference',
    base_framework_version='pytorch1.13',
    instance_type = instance_type
)

##### モデルの定義
先程 S3 にアップロードしたアーティファクトの tar.gz の URI と、コンテナイメージの URI, ロールを設定します。

In [ ]:
huggingface_model = HuggingFaceModel(
    model_data = model_s3_uri,
    role = role,
    image_uri = image_uri
)

デプロイ

In [ ]:
predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type=instance_type,
    endpoint_name=endpoint_name,
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer()
)

#### 推論

##### promptについて
[japanese-gpt-neox-3.6b-instruction-ppo#io-format](https://huggingface.co/rinna/japanese-gpt-neox-3.6b-instruction-ppo#io-format) にある通り、以下の通りにすると良い結果が得られやすいです。  

* プロンプトはユーザーとシステムの会話形式で与える
* 各発言は、以下形式に則る  
    `{ユーザー, システム} : {発言}`
* プロンプトの末尾は`システム:` で終了させる
* 改行は`<NL>`を利用し、発言はすべて `<NL>` で区切る必要がある

以下はプロンプトの例です。`<NL>`の埋め込みが大変なので、改行で書いて後で置換します。

In [ ]:
prompt = '''ユーザー: 世界自然遺産を列挙してください。
システム: 膨大な数です。例えば国で絞ってください。
ユーザー: イギリスでお願いします。
システム:'''.replace('\n','<NL>')
print(prompt)

##### 推論リクエスト
model_fn の実行に時間がかかってしまい、エンドポイントが IN_SERVICE になっても、初回推論はしばらく動かないことがあります。  
CloudWatch Logs に以下のような表示がある場合はしばらく待てば使えるようになります。  
`[WARN] pool-3-thread-1 com.amazonaws.ml.mms.metrics.MetricCollector - worker pid is not available yet.`  
モデルがロードされるまで 6 分程度かかるため、リトライを入れています。
実際の推論時間は 6 秒程度です。

In [ ]:
from time import sleep
request = {
    'prompt' : prompt,
    'max_new_tokens' : 128,
    'do_sample' : True,
    'temperature' : 0.01,
}

for i in range(10):
    try:
        output = predictor.predict(request)['result']
        break
    except:
        sleep(60)
print(output)

##### エンドポイントの削除

In [ ]:
predictor.delete_model()
predictor.delete_endpoint()

### Boto3 を用いる場合
標準だと SageMaker SDK が入っていない環境からデプロイや推論する場合(例:AWS Lambda など)は、boto3 でデプロイや推論することも多いです。  
以下のセルは boto3 で実行する方法を記述しています。
各 API の詳細は Document を確認してください。  
[SageMaker](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html)  
[SageMakerRuntime](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker-runtime.html)  

#### Hosting
##### モデルの作成

In [ ]:
response = sm.create_model(
    ModelName=model_name,
    PrimaryContainer={
        'Image': image_uri,
        'ModelDataUrl': model_s3_uri,
        'Environment': {
            'SAGEMAKER_CONTAINER_LOG_LEVEL': '20',
            'SAGEMAKER_REGION': region,
        }
    },
    ExecutionRoleArn=role,
)

##### エンドポイントコンフィグの作成

In [ ]:
response = sm.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            'VariantName': 'AllTrafic',
            'ModelName': model_name,
            'InitialInstanceCount': 1,
            'InstanceType': 'ml.g5.2xlarge',
        },
    ]
)

##### エンドポイントの作成

In [ ]:
response = sm.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name,
)
endpoint_inservice_waiter.wait(
    EndpointName=endpoint_name,
    WaiterConfig={'Delay': 5,}
)

#### 推論

In [ ]:
# prompt 確認
print(request)

In [ ]:
%%time

# 推論
for i in range(10):
    try:
        response = smr.invoke_endpoint(
            EndpointName=endpoint_name,
            ContentType='application/json',
            Accept='application/json',
            Body=json.dumps(request)
        )
        break
    except:
        sleep(60)
output = json.loads(response['Body'].read().decode('utf-8'))['result']
print(output)

#### お片付け

In [ ]:
sm.delete_endpoint(EndpointName=endpoint_name)
sm.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sm.delete_model(ModelName=model_name)